# MNIST 3.0. - convolutional

This notebook is part of the series using Keras to follow the tutorial ["Tensorflow and deep learning - without a PhD"](https://github.com/martin-gorner/tensorflow-mnist-tutorial). The original tensorflow implementation can be found [here](https://github.com/martin-gorner/tensorflow-mnist-tutorial/blob/master/mnist_3.0_convolutional.py).

Neural network structure for this sample:

```


 · · · · · · · · · ·    (input data, 1-deep)           X [batch, 28, 28, 1]
 @ @ @ @ @ @ @ @ @ @ -- conv. layer 5x5x1=>4 stride 1  W1 [5, 5, 1, 4]        B1 [4]
 ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                   Y1 [batch, 28, 28, 4]
   @ @ @ @ @ @ @ @   -- conv. layer 5x5x4=>8 stride 2  W2 [5, 5, 4, 8]        B2 [8]
   ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                     Y2 [batch, 14, 14, 8]
     @ @ @ @ @ @     -- conv. layer 4x4x8=>12 stride 2 W3 [4, 4, 8, 12]       B3 [12]
     ∶∶∶∶∶∶∶∶∶∶∶                                       Y3 [batch, 7, 7, 12] 
                                                          => reshaped to YY [batch, 7*7*12]
      \x/x\x\x/      -- fully connected layer (relu)   W4 [7*7*12, 200]       B4 [200]
       · · · ·                                         Y4 [batch, 200]
       \x/x\x/       -- fully connected layer (softmax)W5 [200, 10]           B5 [10]
        · · ·                                          Y [batch, 20]
```

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Convolution2D, Reshape

import numpy as np
np.random.seed(123)  # for reproducibility
from keras.utils import np_utils

from keras.optimizers import Adam

Using TensorFlow backend.


Load and preprocess data

In [2]:
from keras.datasets import mnist

# Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocess input data
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Preprocess class labels
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

Define model architecture

In [9]:
K = 4   # first convolutional layer output depth
L = 8   # second convolutional layer output depth
M = 12  # third convolutional layer
N = 200 # fully connected layer

model = Sequential()

stride = 1
model.add(Convolution2D(K, 5, 5, border_mode='same', input_shape=(28, 28, 1), activation='relu', 
                        subsample = (stride,stride)))
stride = 2
model.add(Convolution2D(L, 5, 5, border_mode='same', activation='relu', subsample = (stride,stride)))
stride = 2
model.add(Convolution2D(M, 4, 4, border_mode='same', activation='relu', subsample = (stride,stride)))
model.add(Reshape((7*7*M,)))
model.add(Dense(N, activation='relu'))
model.add(Dense(10, activation='softmax'))

Compile the model

In [5]:
optimizer = Adam(lr=0.003, decay=0.002)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

Fit the model on training data

In [7]:
model.fit(X_train, Y_train, batch_size=1000, nb_epoch=2000, verbose=1)

Evaluate model on test data

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)